## Normalization 归一化/标准化
数据集里的特征往往数值范围差别很大，直接用原始数据，特征值大的数会主导梯度更新，让模型训练不稳定。

Normalization 让不同特征处于相似的数值范围（比如 [0,1] 或均值为 0、方差为 1），避免训练过程不稳定，加快收敛，并提升模型效果。

避免输入过大——梯度爆炸（Loss下降速度非常快后反向放大至NaN）

避免输入过小——梯度消失（Loss几乎不下降）

### 对feature进行归一化（Min-Max Normalization）
如果我们让所有feature的取值范围相同，这样所有训练参数对loss函数的影响就相同了，计算得到的梯度都差不多，就可以用统一的学习率来进行调整了。 

对于bias而言，它的系数为1，相当于它的输入feature大小永远都是1。那么我们就把其他feature都调整到1左右。

最简单的做法，就是让输入feature都除以这个feature的最大值，这样所有feature的取值都是0到1之间

### 对feature进行标准化（（Standardization/Z-score）变成均值为0，标准差为1的分布）

对于归一化，对每个feature除以这个feature所有样本中绝对值最大的值，只有这一个值决定缩放大小。但这个值有可能是个异常值。

相比之下标准化处理会考虑所有样本的分布情况，避免缩放受异常值的影响，训练起来会更稳定。

In [ ]:
"测试是否能用GPU"
import torch
print(torch.__version__)
print(torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

2.8.0+cu128
True
NVIDIA GeForce RTX 5060 Laptop GPU


#### 预测时的归一化
有一点要特别注意，假如你在训练时对数据做了归一化，那么你一定要记录你做归一化时的参数。

在对数据进行预测时，首先需要先对feature用同样的参数进行归一化，然后再带入模型，得到预测值。

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = torch.tensor([[2,1000],[3,2000],[2,500],[1,800],[4,3000]],dtype = torch.float, device = device)
labels = torch.tensor([[19],[31],[14],[15],[43]],dtype = torch.float, device = device)

# 计算特征的均值和标准差
mean = inputs.mean(dim=0)
std = inputs.std(dim=0)

# 标准化
inputs_norm =(inputs-mean)/std

w = torch.ones((2,1),requires_grad=True, device=device)
b = torch.ones((1,),requires_grad=True, device=device)

# 设置超参数
epoch = 1000 # 迭代次数
lr = 0.5    # 学习率

for i in range(epoch):
    outputs = inputs_norm @ w + b  # 前向传播，计算预测值
    loss = torch.mean(torch.square(outputs - labels)) # 计算均方误差MSE损失函数
    print("loss:",loss.item())  # .item()把只含一个数的张量，转换成普通的Python数字标量
    loss.backward()  # 反向传播，计算梯度，存储到 w.grad, b.grad
    print("w.grad:",w.grad.tolist())   
    """
    .tolist()是把任意形状的张量,转成Python列表
    .item()是把只含一个数的张量,转换成普通的Python数字标量
    """
    with torch.no_grad():
        w -= lr * w.grad  # 更新权重
        b -= lr * b.grad  # 更新偏置
        
    w.grad.zero_()
    b.grad.zero_()
    
"""预测新样本,这个预测结果就是模型对新输入new_input 的 pred 值预测"""
new_input = torch.tensor([[3,2500]],dtype = torch.float, device = device)
# 对新样本进行标准化
new_input_norm = (new_input - mean)/std
# 带入模型进行预测
pred = new_input_norm @ w + b
print("预测结果:",pred.item()) 

loss: 635.2097778320312
w.grad: [[-15.60400390625], [-16.726499557495117]]
loss: 25.922632217407227
w.grad: [[9.087748527526855], [8.043952941894531]]
loss: 8.4130220413208
w.grad: [[-4.0536723136901855], [-5.024291038513184]]
loss: 3.343095541000366
w.grad: [[2.8564584255218506], [1.9538871049880981]]
loss: 1.7868238687515259
w.grad: [[-0.8548362255096436], [-1.6941308975219727]]
loss: 1.2350934743881226
w.grad: [[1.0655667781829834], [0.2851123511791229]]
loss: 0.980950653553009
w.grad: [[0.005011647939682007], [-0.7207277417182922]]
loss: 0.8237902522087097
w.grad: [[0.5270583629608154], [-0.14780157804489136]]
loss: 0.7054709792137146
w.grad: [[0.21328718960285187], [-0.4142606258392334]]
loss: 0.6080939769744873
w.grad: [[0.3450268507003784], [-0.2385251522064209]]
loss: 0.5252779126167297
w.grad: [[0.24310298264026642], [-0.299537718296051]]
loss: 0.4540571868419647
w.grad: [[0.26724934577941895], [-0.23734889924526215]]
loss: 0.39258116483688354
w.grad: [[0.2266899198293686], [-

Normalization不会影响深度学习模型的训练结果，因为它仅对参数空间进行了可逆的线性变换，保留了所有必要的信息。